In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb
import pickle
from pathlib import Path as p
import requests as r
from ilxutils.ttl2pd import *
import difflib
from ilxutils.tools import *

In [ ]:
p.home() / 'tests.txt'

In [ ]:
'''Need to open nda types and see if they are subjs in the rdflib nif ontologies'''

In [ ]:
args = read_args(api_key= p.home() / 'keys/production_api_scicrunch_key.txt', db_url= p.home() / 'keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(db_url=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, db_url=args.db_url)

In [ ]:
cp(annos, '../../../ilx_db_anno_backup') #need to make a script to back up all tables
cp(ex, '../../../ilx_db_ex_backup')
cp(terms, '../../../ilx_db_terms_backup')

In [ ]:
#annos = op('../../../ilx_db_anno_backup')
#ex = op('../../../ilx_db_ex_backup')
terms = op('../../../ilx_db_terms_backup')

In [ ]:
terms = sql.get_terms()
annos = sql.get_annotations()
ex = sql.get_existing_ids()

In [ ]:
for value in ['condition', 'filterElement', 'size', 'notes', 'id', 'valueRange', 'required', 'type']:
    if not annos[(annos.annotation_label == 'hasConstraint') & (annos.value.str.contains(value+' :'))].empty:
        print(value)

In [ ]:
annos[annos.id == 6909]

In [ ]:
terms[terms.id == 15077]

In [ ]:
annos[annos.value.str.contains('type : File')]

In [ ]:
terms[terms.id == 53165]

In [ ]:
for i, row in terms.iterrows():
    if row.type not in ['annotation', 'term', 'relationship', 'cde']:
        print(row, 'here')

In [ ]:
annos[annos.value.str.contains('type : File')]

In [ ]:
terms[terms.label == 'grade_highed (AASPIRE Health Integration)']

In [ ]:
annos.head()

In [ ]:
ex.head()

In [ ]:
set(annos[annos.annotation_label.str.contains('type')].annotation_label)

In [ ]:
'''check if all 2nd tier has NDA.CDE exs'''
excheck = defaultdict(list)
for i, row in ex.iterrows():
    excheck[row.tid].append(row.curie.split(':', 1)[0])

In [ ]:
for tid in oj('../../../nda_tier2_term_ids'):
    if 'NDA.CDE' not in excheck[tid]:
        print(tid, excheck[tid])
        break

In [ ]:
for i, row in annos.iterrows():
    print({**row, **{'test':1}})
    break

In [ ]:
nda_tier2_term_ids = []
for i, row in annos[annos.annotation_label == 'hasConstraint'].iterrows(): ## get second tier annotations only -> anno_label == 'hasConstraint'
    key_type, key_val = contains_key(row.value)
    if key_val:
        nda_tier2_term_ids.append(row.tid)
cj(nda_tier2_term_ids, '../../../nda_tier2_term_ids')

In [ ]:
terms[terms.id == 304437]

In [ ]:
stak = { #second tier annotation keys
    'condition':True,
    'filterElement':True,
    'id':True,
    'notes':True,
    'required':True,
    'size':True,
    'type':True,
    'valueRange':True,
}
def contains_key(string):
    if ':' in string:
        sk, sval = string.split(':', 1)[0].strip(), string.split(':', 1)[1].strip() #string key
        if stak.get(sk):
            return sk, sval
    return False, False

udata = []
adata = []
cdata = []
ddata = []
tudata = []
visited = {}
filterElementID = 304437
required = 304438
condition = 304439
size = 304440
minValue = 304441
maxValue = 304442
allowedTypes = 304443
allowedValues = 304444
for i, row in annos[annos.annotation_label == 'hasConstraint'].iterrows(): ## get second tier annotations only -> anno_label == 'hasConstraint'
    if i % 200000 == 0: print(i)
    key_type, key_val = contains_key(row.value) ## filter each term by their annotation values by key : value
    
#     if key_type == 'condition':
#         if key_val == "None":
#             udata.append({'id':row['id'], 'tid':row['tid'], 'annotation_tid':condition, 'value':key_val})        
#         else:
#             print(key_type, row.ilx, row.label)
    
#     elif key_type == 'filterElement':
#         if key_val == "None":
#             udata.append({'id':row['id'], 'tid':row['tid'], 'annotation_tid':filterElementID, 'value':'False'})
#         elif key_val == '1':
#             udata.append({'id':row['id'], 'tid':row['tid'], 'annotation_tid':filterElementID, 'value':'True'})
#         else:
#             sys.exit(key_val)
            
#     elif key_type == 'id':
#        ddata.append(row.id)
        
#     elif key_type == 'notes':
#        if key_val != "None" and not visited.get(row.tid):
#            tudata.append({'id':row.tid, 'ilx':row.term_ilx, 'comment':key_val})
#            visited[row.tid] = True
#        ddata.append(row.id)
    
#     elif key_type == 'required':
#         if key_val == "None":
#             udata.append({'id':row['id'], 'tid':row['tid'], 'annotation_tid':required, 'value':key_val})
#         else:
#             print(key_type, row.ilx, row.label)
      
#     elif key_type == 'size':
#         if key_val == "None":
#             udata.append({'id':row['id'], 'tid':row['tid'], 'annotation_tid':size, 'value':key_val})
#         else:
#             print(key_type, row.ilx, row.label)
    

    if key_type == 'type':#{'Date', 'File', 'Float', 'GUID', 'Integer', 'String', 'Thumbnail'}
        memo = {'id':row['id'], 'tid':row['tid'], 'annotation_tid':allowedTypes, 'value':None}
        pairs = {
            'Date':'xsd:dateTime',
            'File':'xsd:string',
            'Float':'xsd:float',
            'GUID':'ilxtr:guid',
            'Integer':'xsd:integer',
            'String':'xsd:string',
            'Thumbnail':'xsd:string',
        }
        if pairs.get(key_val):
            memo['value'] = pairs[key_val]
        else:
            sys.exit('type fail') 
        udata.append(memo)
    
#     if key_type == 'valueRange':
#         if key_val == 'None': 
#             udata.append({'id':row['id'], 'tid':row.tid, 'annotation_tid':allowedValues, 'value':key_val})
#         else:
#             print(key_type, row.ilx, row.label)
    
        #if '::' in key_val:
        #    if len(key_val.split('::')) == 2:
        #        _minValue, _maxValue = key_val.split('::')[0].strip(), key_val.split('::')[1].strip()
        #        ddata.append(row.id)
        #        adata.append({'id':row['id'], 'tid':row.tid, 'annotation_tid':minValue, 'value':_minValue})
        #        adata.append({'id':row['id'], 'tid':row.tid, 'annotation_tid':maxValue, 'value':_maxValue})
        #    else:
        #        udata.append({'id':row['id'], 'tid':row.tid, 'annotation_tid':allowedValues, 'value':key_val})
        #else:
        #    udata.append({'id':row['id'], 'tid':row.tid, 'annotation_tid':allowedValues, 'value':key_val})

## condition -> ?
## filterElement -> ?
## delete Id if it is preferred already
## notes -> move to notes
## required -> get list of options this has
## size -> get ex that isn't none
## type -> porb
## valueRange -> seperate into minValue and maxValue

In [ ]:
#cj(sorted(ddata), '../../../nda_annotation_valueRange_delete')
cj(udata, '../../../nda_type_update')
#cj(adata, '../../../nda_minmaxvalue_add')
#cj(tudata, '../../../nda_term_update')
#cj(udata, '../../../nda_annotation_update')
#cj(cdata, '../../../nda_annotation_check')
#cj(udata, '../../../nda_None_update')

In [ ]:
data = [{'id':304444, 'label':'allowedValues', 'type':'annotation'}]
output = sci.updateTerms(data, LIMIT=1, crawl=True, _print=False)#304437
output

In [ ]:
cdata = oj('../../../nda_annotation_check')

In [ ]:
'''explainer'''
ctypes = defaultdict(set)
for d in cdata:
    ctypes[list(d)[0]].add(list(d.values())[0])

In [ ]:
#ctypes['codition'] #random code; ex ->  'famhx_1   == 1 &&  fam_history_12_yes_no   == 1 &&  fhx_3h_younger_p  > 4 &&  fhx_3h_younger_p  != 999',
#ctypes['size'] #ints with no measurement specifics 
#ctypes['filterElement'] #either 1 or None, makes me think its really 0 or 1 for true or false
#ctypes['required'] # {'Conditional', 'Optional', 'Recommended', 'Required'}
#ctypes['type'] #{'Date', 'File', 'Float', 'GUID', 'Integer', 'String', 'Thumbnail'}
#ctypes['valueRange']

In [ ]:
import subprocess as sb
for typ in ['Date', 'File', 'Float', 'GUID', 'Integer', 'String', 'Thumbnail']:
    datatype = "xsd:"+typ
    sbinput="grep -nir '{}' /home/troy/Desktop/work/NIF-Ontology/".format(datatype)
    print(sbinput)
    output = sb.getoutput(sbinput)
    if output and typ not in ['String']:
        print(output)

In [ ]:
from rdflib.namespace import XSD
XSD.troy